### KoSimCSE NLI 학습

In [10]:
from transformers import AutoModel, AutoTokenizer
from bert import BERT
import pandas as pd
from sentence_transformers import losses
import math
from sentence_transformers.readers import InputExample
import random
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
model_path = 'KoSimCSE-roberta-multitask/'
data_path = '../Dataset/kb_dataset_nli.csv'

df_nli = pd.read_csv(data_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [2]:
df_nli = df_nli.drop_duplicates(ignore_index=True)
df_nli

,hyphothesis,label,premise
0,은행에서는 매주 최대 1천만원을 추첨하는 이벤트를 진행하고 있습니다.,entailment,은행에서는 단 3일 동안 최대 1천만원을 추첨하는 럭키박스 이벤트를 진행하고 있습니다.
1,은행에서는 단 하루 동안 최대 1천만원을 추첨하는 럭키박스 이벤트를 진행하고 있습니다.,neutral,은행에서는 단 3일 동안 최대 1천만원을 추첨하는 럭키박스 이벤트를 진행하고 있습니다.
2,은행에서는 단 3일 동안 최대 100만원을 추첨하는 럭키박스 이벤트를 진행하고 있습니다.,contradiction,은행에서는 단 3일 동안 최대 1천만원을 추첨하는 럭키박스 이벤트를 진행하고 있습니다.
3,"이 이벤트는 은행의 VIP 고객에게만 제공되며, 참여하신 분들 중 한 분을 추첨하여...",entailment,"이 이벤트는 은행의 고객에게만 제공되며, 참여하신 분들 중 한 분을 추첨하여 1천만..."
4,"이 이벤트는 은행의 직원에게만 제공되며, 참여하신 분들 중 한 분을 추첨하여 1천만...",neutral,"이 이벤트는 은행의 고객에게만 제공되며, 참여하신 분들 중 한 분을 추첨하여 1천만..."
...,...,...,...
1018,스타뱅킹 길라잡이와 상담하거나 챗봇을 이용하지 않고 문의할 수 있습니다.,neutral,스타뱅킹 길라잡이와 상담하거나 챗봇을 이용하여 문의할 수 있습니다.
1019,스타뱅킹 길라잡이와 상담하거나 챗봇을 이용하여 문의할 수 없습니다.,contradiction,스타뱅킹 길라잡이와 상담하거나 챗봇을 이용하여 문의할 수 있습니다.
1020,은행 고객센터에서 문의하시면 자세한 내용을 안내해드립니다.,entailment,자세한 내용은 은행 고객센터나 홈페이지를 참고해주세요.
1021,은행 고객센터에서는 다른 문제에 대해서도 도움을 받을 수 있습니다.,neutral,자세한 내용은 은행 고객센터나 홈페이지를 참고해주세요.


In [ ]:

def make_nli_triplet_input_example(dataset):
    ''' 
    Transform to Triplet format and InputExample
    ''' 
    # transform to Triplet format
    train_data = {}
    def add_to_samples(sent1, sent2, label):
        if sent1 not in train_data:
            train_data[sent1] = {'contradiction': set(), 'entailment': set(), 'neutral': set()}
        train_data[sent1][label].add(sent2)

    for i, data in dataset.iterrows():
        sent1 = data['hyphothesis'].strip()
        sent2 = data['premise'].strip()
        label = data['label']

        add_to_samples(sent1, sent2, label)
        add_to_samples(sent2, sent1, label) #Also add the opposite

    # transform to InputExmaples
    input_examples = []
    for sent1, others in train_data.items():
        if len(others['entailment']) > 0 and len(others['contradiction']) > 0:
            input_examples.append(InputExample(texts=[sent1, random.choice(list(others['entailment'])), random.choice(list(others['contradiction']))]))
            input_examples.append(InputExample(texts=[random.choice(list(others['entailment'])), sent1, random.choice(list(others['contradiction']))]))
        
    return input_examples



nli_train_examples = make_nli_triplet_input_example(df_nli)

In [4]:
# Train Dataloader
train_dataloader = DataLoader(
    nli_train_examples,
    shuffle=True,
    batch_size=16,
)

In [3]:
# Load Embedding Model
embedding_model = models.Transformer(
    model_name_or_path=model_path, 
    max_seq_length=256,
    do_lower_case=True
)

# Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

In [ ]:
# config
nli_num_epochs = 4
train_batch_size = 32
nli_model_save_path = 'output/training_nli_/'

# Use MultipleNegativesRankingLoss
train_loss = losses.MultipleNegativesRankingLoss(model)
# warmup steps
warmup_steps = math.ceil(len(nli_train_examples) * nli_num_epochs / train_batch_size * 0.1) #10% of train data for warm-up

# Training
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=nli_num_epochs,
    warmup_steps=warmup_steps,
    output_path=nli_model_save_path,
    use_amp=False       #Set to True, if your GPU supports FP16 operations
)

### KoSimCSE STS 추가 학습

In [4]:
data_path = '../Dataset/kb_dataset_sts_all.csv'

df = pd.read_csv(data_path)

In [5]:
nli_model_save_path = 'output/training_nli_/'
model = SentenceTransformer(nli_model_save_path)

In [6]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': True}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [7]:
def make_sts_input_example(dataset):
    ''' 
    Transform to InputExample
    ''' 
    input_examples = []
    for i, data in dataset.iterrows():
        sentence1 = data['sent1']
        sentence2 = data['sent2']
        score = (data['score']) / 5.0  # normalize 0 to 5
        input_examples.append(InputExample(texts=[sentence1, sentence2], label=score))

    return input_examples

 
  
sts_train_examples = make_sts_input_example(df)

In [11]:
# Train Dataloader
train_dataloader = DataLoader(
    sts_train_examples,
    shuffle=True,
    batch_size=16, 
)

In [16]:
len(sts_train_examples)


11220

In [ ]:
sts_num_epochs = 4
train_batch_size = 32
sts_model_save_path = 'output/training_sts_all_continue_training/'

# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)
# warmup steps
warmup_steps = math.ceil(len(sts_train_examples) * sts_num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
# Training
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=sts_num_epochs,
    warmup_steps=warmup_steps,
    output_path=sts_model_save_path
)